In [596]:
import pandas as pd
import numpy as np

## Help Functions

In [597]:
# Functions that will help cleaning the dataset
def get_winner(df, cat=False):
    """
    Get the winner from the dataset based on goals
    """
    if df['home_team_score'] > df['away_team_score']:
        return df['home_team'] if not cat else 1
    return df['away_team'] if not cat else 0

def clean_stage(df):
    """
    Remove the groups x and replace it just with Groups
    """
    if df['Stage'].__contains__('Group'):
        return 'Group'
    return df['Stage']

def mix_fifa_data(df, fifa_data):
    """
    Add fifa data
    """
    df.reset_index(drop=True, inplace=True)
    for i in range(df.shape[0]):
        df['home_overall'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Overall'].values[0].astype(np.float64)
        df['away_overall'][i] = fifa_data[fifa_data['Nationality'] == df['away_team'][i]]['Overall'].values[0].astype(np.float64)
        df['home_age'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Age'].values[0].astype(np.float64)
        df['away_age'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Age'].values[0].astype(np.float64)
        df['home_potential'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Potential'].values[0].astype(np.float64)
        df['away_potential'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Potential'].values[0].astype(np.float64)
        df['home_reputation'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['International Reputation'].values[0].astype(np.float64)
        df['away_reputation'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['International Reputation'].values[0].astype(np.float64)

## Start

In [598]:
# Datos desde el local
international_matches = pd.read_csv("files/international_matches.csv")
data_fifa_23 = pd.read_csv('files/FIFA23_official_data.csv')

In [599]:
#List of all Teams in 2022 World Cup
teams_worldcup = ['Qatar', 'Ecuador', 'Senegal', 'Netherlands', 'England', 'Iran', 'USA',
                  'Wales', 'Argentina', 'Saudi Arabia', 'Mexico', 'Poland', 'France', 
                  'Australia', 'Denmark', 'Tunisia', 'Spain', 'Costa Rica', 'Germany', 
                  'Japan', 'Belgium', 'Canada', 'Morocco', 'Croatia', 'Brazil', 'Serbia', 
                  'Switzerland', 'Cameroon', 'Portugal', 'Ghana', 'Uruguay', 'Korea Republic']

## Players Data

In [600]:
# Elimino Columnas a ojo
data_fifa_23.drop(['ID', 'Photo', 'Flag', 'Club', 'Club Logo', 'Release Clause', 'Kit Number', 'Contract Valid Until',
                   'Real Face', 'Body Type', 'Work Rate', 'Position', 'Loaned From', 'Name', 'Preferred Foot', 'Wage', 'Weak Foot'], axis=1, inplace=True)

In [601]:
data_fifa_23.replace({'United States': 'USA'}, inplace=True)

In [602]:
data_fifa_grouped = data_fifa_23.groupby(['Nationality']).mean()

/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/3576440808.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_fifa_grouped = data_fifa_23.groupby(['Nationality']).mean()


In [603]:
data_fifa_grouped.reset_index(inplace=True)

In [604]:
data_fifa_grouped = data_fifa_grouped.reset_index(level=0)

## Matches International

In [605]:
# Turn date into a datetime object
international_matches['date'] = pd.to_datetime(international_matches['date'])
international_matches['home_overall'] = np.float64(0)
international_matches['away_overall'] = np.float64(0)
international_matches['home_age'] = np.float64(0)
international_matches['away_age'] = np.float64(0)
international_matches['home_potential'] = np.float64(0)
international_matches['away_potential'] = np.float64(0)
international_matches['home_reputation'] = np.float64(0)
international_matches['away_reputation'] = np.float64(0)

In [606]:
international_matches = international_matches.loc[(international_matches['home_team'].isin(teams_worldcup) | international_matches['away_team'].isin(teams_worldcup)) & (international_matches['date'] > '2018-01-01')]
international_matches['Won'] = international_matches.apply(lambda row: get_winner(row, cat=True), axis=1)
mix_fifa_data(international_matches, data_fifa_grouped)
international_matches.drop(['home_team_continent', 'date', 'away_team_continent', 'shoot_out', 'home_team_result', 'tournament', 'city', 'neutral_location', 'country'], inplace=True, axis=1)

/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/3884684.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['home_overall'][i] = fifa_data[fifa_data['Nationality'] == df['home_team'][i]]['Overall'].values[0].astype(np.float64)
/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/3884684.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['away_overall'][i] = fifa_data[fifa_data['Nationality'] == df['away_team'][i]]['Overall'].values[0].astype(np.float64)
/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/3884684.py:26: SettingWithCopyWarning: 
A value is trying to be s

In [607]:
# Country group by
country_gb = international_matches.groupby(['home_team']).mean()

/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/125984719.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_gb = international_matches.groupby(['home_team']).mean()


In [608]:
international_matches.loc[international_matches['away_team'] == 'Qatar', "away_team_goalkeeper_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['away_team'] == 'Tunisia', "away_team_goalkeeper_score"] = country_gb['home_overall']['Tunisia']
international_matches.loc[international_matches['home_team'] == 'Qatar', "home_team_goalkeeper_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['away_team'] == 'Qatar', "away_team_mean_defense_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['home_team'] == 'Qatar', "home_team_mean_defense_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['away_team'] == 'Qatar', "away_team_mean_midfield_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['home_team'] == 'Qatar', "home_team_mean_midfield_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['away_team'] == 'Qatar', "away_team_mean_offense_score"] = country_gb['home_overall']['Qatar']
international_matches.loc[international_matches['home_team'] == 'Qatar', "home_team_mean_offense_score"] = country_gb['home_overall']['Qatar']

# Country group by with the updated not null values
country_gb = international_matches.groupby(['home_team']).mean()

/var/folders/kw/rsgvj_hn27153sygy82j1bl0bl35zf/T/ipykernel_27812/1897047855.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_gb = international_matches.groupby(['home_team']).mean()


In [609]:
international_matches.head()

,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,...,away_team_mean_midfield_score,home_overall,away_overall,home_age,away_age,home_potential,away_potential,home_reputation,away_reputation,Won
0,Germany,Spain,1,6,1609,1228,1,1,92.0,90.0,...,87.0,63.504817,67.364646,22.714836,22.714836,71.285164,71.285164,1.137765,1.137765,0
1,Netherlands,England,21,16,963,1046,0,1,81.0,82.0,...,82.8,65.114286,61.084259,22.002198,22.002198,73.907692,73.907692,1.171429,1.171429,0
2,Serbia,Morocco,34,42,779,693,1,2,74.0,71.0,...,80.2,67.060345,67.344828,25.215517,25.215517,72.103448,72.103448,1.189655,1.189655,0
3,Portugal,Netherlands,3,21,1360,963,0,3,83.0,81.0,...,82.8,68.553254,65.114286,23.502959,23.502959,75.538462,75.538462,1.192308,1.192308,0
4,Wales,Uruguay,20,22,983,931,0,1,74.0,82.0,...,78.0,61.029197,65.011990,22.102190,22.102190,70.145985,70.145985,1.087591,1.087591,0


## Model

In [610]:
from sklearn import linear_model, svm, tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

In [611]:
data = international_matches.drop(['home_team', 'away_team'], axis=1)

X = data.drop(['Won'], axis=1)
y = data['Won']

# X columns
x_cols = list(X.columns)

# Scaling
scaler = StandardScaler()
scaler.fit(X)
X_s = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_s, y)

In [621]:
params = {
    "solver": ("saga", ),
    "penalty": ("l1", "l2", "elasticnet", "none"),
    "l1_ratio": (1,)
}
gd = GridSearchCV(linear_model.LogisticRegression(max_iter=100000), param_grid=params, scoring="accuracy")
gd.fit(X_train, y_train)
y_pred = gd.best_estimator_.predict(X_test)

confusion_matrix(y_test, y_pred)

/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_rat

array([[44,  0],
       [ 0, 31]])

## Predict

In [ ]:
international_matches.head()

,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,...,away_team_mean_midfield_score,home_overall,away_overall,home_age,away_age,home_potential,away_potential,home_reputation,away_reputation,Won
0,Germany,Spain,1,6,1609,1228,1,1,92.0,90.0,...,87.0,63.504817,67.364646,22.714836,22.714836,71.285164,71.285164,1.137765,1.137765,0
1,Netherlands,England,21,16,963,1046,0,1,81.0,82.0,...,82.8,65.114286,61.084259,22.002198,22.002198,73.907692,73.907692,1.171429,1.171429,0
2,Serbia,Morocco,34,42,779,693,1,2,74.0,71.0,...,80.2,67.060345,67.344828,25.215517,25.215517,72.103448,72.103448,1.189655,1.189655,0
3,Portugal,Netherlands,3,21,1360,963,0,3,83.0,81.0,...,82.8,68.553254,65.114286,23.502959,23.502959,75.538462,75.538462,1.192308,1.192308,0
4,Wales,Uruguay,20,22,983,931,0,1,74.0,82.0,...,78.0,61.029197,65.011990,22.102190,22.102190,70.145985,70.145985,1.087591,1.087591,0


In [ ]:
international_matches.columns

Index(['home_team', 'away_team', 'home_team_fifa_rank', 'away_team_fifa_rank',
       'home_team_total_fifa_points', 'away_team_total_fifa_points',
       'home_team_score', 'away_team_score', 'home_team_goalkeeper_score',
       'away_team_goalkeeper_score', 'home_team_mean_defense_score',
       'home_team_mean_offense_score', 'home_team_mean_midfield_score',
       'away_team_mean_defense_score', 'away_team_mean_offense_score',
       'away_team_mean_midfield_score', 'home_overall', 'away_overall',
       'home_age', 'away_age', 'home_potential', 'away_potential',
       'home_reputation', 'away_reputation', 'Won'],
      dtype='object')

In [ ]:
country_gb

,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,...,away_team_mean_midfield_score,home_overall,away_overall,home_age,away_age,home_potential,away_potential,home_reputation,away_reputation,Won
home_team,,,,,,,,,,,,,,,,,,,,,
Argentina,8.727273,19.272727,1596.090909,1530.636364,1.636364,0.545455,82.000000,80.272727,81.718182,90.018182,...,81.145455,66.844603,65.271353,24.321471,24.321471,73.293001,73.293001,1.125741,1.125741,0.636364
Australia,37.666667,41.666667,1469.000000,1460.666667,0.333333,1.000000,77.333333,72.333333,72.400000,71.433333,...,76.033333,58.257009,57.385986,21.995327,21.995327,66.640187,66.640187,1.014019,1.014019,0.000000
Belgium,2.176471,17.647059,1712.352941,1488.352941,2.529412,1.117647,89.117647,80.411765,82.176471,86.676471,...,80.494118,64.154135,63.628390,21.721805,21.721805,73.992481,73.992481,1.199248,1.199248,0.764706
Brazil,2.933333,24.066667,1737.400000,1532.866667,1.533333,0.533333,86.800000,79.928571,86.120000,87.526667,...,79.457143,70.556586,64.530203,25.417440,25.417440,74.729128,74.729128,1.274583,1.274583,0.600000
Cameroon,51.000000,50.000000,1404.000000,1423.000000,0.000000,0.000000,78.000000,69.000000,77.200000,79.000000,...,77.800000,66.554054,65.422414,24.351351,24.351351,72.297297,72.297297,1.202703,1.202703,0.000000
Canada,52.750000,21.500000,1412.000000,1576.000000,1.750000,0.250000,75.500000,80.250000,68.850000,73.000000,...,75.000000,61.513158,62.788981,22.605263,22.605263,70.539474,70.539474,1.065789,1.065789,1.000000
Costa Rica,43.125000,28.750000,1372.875000,1443.500000,0.625000,1.000000,87.125000,77.428571,72.000000,69.712500,...,78.114286,67.733333,63.409433,27.000000,27.000000,70.666667,70.666667,1.266667,1.266667,0.250000
Croatia,10.583333,10.750000,1582.583333,1617.666667,1.500000,1.750000,82.833333,82.083333,80.116667,79.733333,...,82.741667,68.216049,65.440396,24.765432,24.765432,73.635802,73.635802,1.203704,1.203704,0.333333
Denmark,12.100000,13.500000,1445.100000,1442.200000,1.000000,0.600000,83.800000,82.600000,79.540000,77.540000,...,81.660000,62.700441,63.418818,21.916300,21.916300,72.343612,72.343612,1.083700,1.083700,0.400000


In [ ]:
X

array([[-1.00951792, -0.79274282,  0.36242331, ..., -0.18978415,
         0.09469347,  0.09469347],
       [ 0.13937015, -0.30934327, -2.08005475, ...,  0.85455161,
         0.51878905,  0.51878905],
       [ 0.88614739,  0.94749556, -2.7757451 , ...,  0.13607075,
         0.74840835,  0.74840835],
       ...,
       [-0.49251829, -0.21266336,  0.54768867, ...,  0.85455161,
         0.51878905,  0.51878905],
       [ 0.42659216, -0.98610264,  0.11666313, ..., -0.60079001,
        -0.87270083, -0.87270083],
       [ 0.25425895,  0.60911588,  0.15069146, ..., -1.15139622,
        -1.10762347, -1.10762347]])

In [595]:
country_gb[country_gb['home_team_mean_midfield_score'].isnull()]

,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,...,away_team_mean_midfield_score,home_overall,away_overall,home_age,away_age,home_potential,away_potential,home_reputation,away_reputation,Won
home_team,,,,,,,,,,,,,,,,,,,,,
Qatar,57.6,34.6,1390.2,1495.8,1.0,2.6,NaN,77.8,NaN,NaN,...,79.6,66.862069,64.115919,27.0,27.0,69.586207,69.586207,1.0,1.0,0.2


In [623]:
while True:
    home_team = input("Home Team: ")
    away_team = input("Away Team: ")

    if home_team in teams_worldcup and away_team in teams_worldcup:
        break

columns = ['team_fifa_rank', 'team_total_fifa_points','team_score', 'team_goalkeeper_score', 'team_mean_defense_score','team_mean_offense_score', 'team_mean_midfield_score', 'overall', 'age', 'potential', 'reputation']
data_dict = {}
for column in columns:
    data_dict['home_' + column] = country_gb['home_' + column][home_team]
    data_dict['away_' + column] = country_gb['away_' + column][away_team]


predict_df = pd.DataFrame([data_dict])
result = gd.best_estimator_.predict(predict_df)

winner = home_team if result else away_team
print(f"{home_team} VS. {away_team} winner is: {winner} -- {result}")

Germany VS. Qatar winner is: Qatar -- [0]


/Users/crestrepo/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [624]:
data_dict

{'home_team_fifa_rank': 10.466666666666667,
 'away_team_fifa_rank': 34.6,
 'home_team_total_fifa_points': 1590.0666666666666,
 'away_team_total_fifa_points': 1495.8,
 'home_team_score': 1.3333333333333333,
 'away_team_score': 2.6,
 'home_team_goalkeeper_score': 90.53333333333333,
 'away_team_goalkeeper_score': 77.8,
 'home_team_mean_defense_score': 84.82,
 'away_team_mean_defense_score': 77.8,
 'home_team_mean_offense_score': 84.01333333333334,
 'away_team_mean_offense_score': 79.72,
 'home_team_mean_midfield_score': 86.88000000000001,
 'away_team_mean_midfield_score': 79.6,
 'home_overall': 63.50481695568401,
 'away_overall': 64.11591944480408,
 'home_age': 22.714836223506744,
 'away_age': 27.0,
 'home_potential': 71.28516377649326,
 'away_potential': 69.58620689655173,
 'home_reputation': 1.1377649325626205,
 'away_reputation': 1.0}